In [1]:
from pandas import read_csv
import cv2
import glob
import os
import numpy as np
import logging
import coloredlogs
logger = logging.getLogger(__name__)
coloredlogs.install(level='DEBUG')
coloredlogs.install(level='DEBUG', logger=logger)

In [2]:
IM_EXTENSIONS = ['png', 'jpg', 'jpeg', 'bmp']

In [3]:
def read_img(img_path, img_shape=(128, 128)):
    """
    load image file and divide by 255.
    """
    img = cv2.imread(img_path)
    img = cv2.resize(img, img_shape)
    img = img.astype('float')
    img /= 255.

    return img

In [115]:
dataset_dir = './data/images/'
label_path = './data/label.csv'
batch_size=32,
img_shape=(128, 128)

In [116]:
label_df = read_csv(label_path)
# img_files = glob.glob(dataset_dir + '*')
# img_files = [f for f in img_files if f[-3:] in IM_EXTENSIONS]

label_idx = label_df.set_index('filename')
img_files = label_idx.index.unique().values

In [117]:
label_idx.loc['0_Parade_Parade_0_628.jpg'].head()

,xmin,ymin,xmax,ymax,class
filename,,,,,
0_Parade_Parade_0_628.jpg,26,299,36,315,face
0_Parade_Parade_0_628.jpg,25,329,32,340,face
0_Parade_Parade_0_628.jpg,84,341,90,349,face
0_Parade_Parade_0_628.jpg,80,329,85,335,face
0_Parade_Parade_0_628.jpg,110,335,114,344,face


In [118]:
label_idx.iloc[0:5]

,xmin,ymin,xmax,ymax,class
filename,,,,,
0_Parade_Parade_0_628.jpg,26,299,36,315,face
0_Parade_Parade_0_628.jpg,25,329,32,340,face
0_Parade_Parade_0_628.jpg,84,341,90,349,face
0_Parade_Parade_0_628.jpg,80,329,85,335,face
0_Parade_Parade_0_628.jpg,110,335,114,344,face


In [119]:
len(img_files)

2989

In [120]:
def append_zero(arr):
    return np.append([0], arr)

In [121]:
# temp = label_idx.loc[img_files[0]].values[:, :4] #[0, 26, 299, 36, 315]
# np.apply_along_axis(append_zero, 1, temp)

In [194]:
"""
data loader

return image, [class_label, class_and_location_label]
"""

numofData = len(img_files)  # endwiths(png,jpg ...)
data_idx = np.arange(numofData)

while True:
    batch_idx = np.random.choice(data_idx, size=batch_size, replace=False)

    batch_img = []
    batch_label = []
    batch_label_cls = []

    for i in batch_idx:

        img = read_img(dataset_dir + img_files[i], img_shape=img_shape)
        label_idx = label_df.set_index('filename')
        img_files = label_idx.index.unique().values
        label = label_idx.loc[img_files[i]].values
        label = np.array(label, ndmin=2)
        label = label[:, :4]
        cls_loc_label = np.apply_along_axis(append_zero, 1, label)
        batch_img.append(img)
        batch_label.append(label)
        batch_label_cls.append(0)  # label[0:1]) ---> face
#     yield ({'input_1': np.array(batch_img, dtype=np.float32)},
#            {'clf_output': np.array(batch_label_cls, dtype=np.float32),
#                          'bb_output': np.array(batch_label, dtype=np.float32)})


[[0 244 104 306 191]
 [0 317 425 381 501]
 [0 490 313 558 406]
 [0 641 90 702 157]]


In [310]:
import tensorflow as tf

In [469]:
def dataloader(dataset_dir, label_path,  batch_size=1000, img_shape=(128, 128)):
    """
    data loader

    return image, [class_label, class_and_location_label]
    """

    label_df = read_csv(label_path)
    label_idx = label_df.set_index('filename')
    img_files = label_idx.index.unique().values

    numofData = len(img_files)  # endwiths(png,jpg ...)
    data_idx = np.arange(numofData)

    while True:
        batch_idx = np.random.choice(data_idx, size=batch_size, replace=False)

        batch_img = []
        batch_label = []
        batch_class = []

        for i in batch_idx:

            img = read_img(dataset_dir + img_files[i], img_shape=img_shape)

            label = label_idx.loc[img_files[i]].values
            label = np.array(label, ndmin=2)
            label = label[:, :4]

            cls_loc_label = np.apply_along_axis(append_zero, 1, label)

            batch_img.append(img)
            batch_label.append(cls_loc_label)   # face + bb
            batch_class.append(cls_loc_label[:, 0:1])  # label[:, 0:1]) ---> face

#         yield {'input_1': np.array(batch_img, dtype=np.float32)}, {'clf_output': np.array(batch_class, dtype=np.float32),'bb_output': np.array(batch_label, dtype=np.float32)}
            
        yield np.array(batch_img, dtype=np.float32), [np.array(batch_class, dtype=np.float32), np.array(batch_label, dtype=np.float32)]

In [470]:
data_gen = dataloader(dataset_dir, label_path,  batch_size=1, img_shape=(128, 128))

In [471]:
data = next(data_gen)

In [472]:
len(data)

2